In [6]:
import os
import json
import numpy as np
from tqdm import tqdm_notebook

magazine_path = '/data/private/Arena/datasets/magazine.json'
metadata_path = '/data/private/Arena/datasets/metadata.json'
users_path = '/data/private/Arena/datasets/users.json'
predict_path = '/data/private/Arena/datasets/predict/'
read_path = '/data/private/Arena/datasets/read/'

In [5]:
#magazine -> keyword list
magazine_list = []
for line in open(magazine_path, 'r', encoding='utf-8'):
    magazine_list.append(json.loads(line))
metadata_list = []
for line in open(metadata_path, 'r', encoding='utf-8'):
    metadata_list.append(json.loads(line))
users_list = []
for line in open(users_path, 'r', encoding='utf-8'):
    users_list.append(json.loads(line))

In [3]:
magazine_list[0], metadata_list[0], users_list[0]

({'magazine_tag_list': ['브런치북', '육아일기', '대화법', '들려주고픈이야기'], 'id': '38842'},
 {'magazine_id': 8982,
  'user_id': '@bookdb',
  'title': '사진으로 옮기기에도 아까운, 리치필드 국립공원',
  'keyword_list': ['여행', '호주', '국립공원'],
  'display_url': 'https://brunch.co.kr/@bookdb/782',
  'sub_title': '세상 어디에도 없는 호주 Top 10',
  'reg_ts': 1474944427000,
  'article_id': 782,
  'id': '@bookdb_782'},
 {'keyword_list': [],
  'following_list': ['@perytail', '@brunch'],
  'id': '#901985d8bc4c481805c4a4f911814c4a'})

In [2]:
for line in open('./datasets/predict/dev.users', 'r', encoding='utf-8'):
    print(line)
    break

#d6866a498157771069fdf15361cb012b



In [ ]:
keyword_list = []
for data in tqdm_notebook(magazine_list):
    for keyword in data['magazine_tag_list']:
        if keyword not in keyword_list:
            keyword_list.append(keyword)
for data in tqdm_notebook(metadata_list):
    for keyword in data['keyword_list']:
        if keyword not in keyword_list:
            keyword_list.append(keyword)
for data in tqdm_notebook(users_list):
    for keyword in data['keyword_list']:
        if keyword not in keyword_list:
            keyword_list.append(keyword)

In [59]:
item2keywd = {0:['없음']}
item_list = ['unk']
item_dict = {'unk':0}

for data in tqdm_notebook(metadata_list):
    if item_dict.get(data['id']) == None:
        item_dict[data['id']] = len(item_list)
        item_list.append(data['id'])
    if data['keyword_list']:
        item2keywd[data['id']] = data['keyword_list']
    else:
        item2keywd[data['id']] = ['없음']
        
np.save('/data/private/Arena/prepro_results/item_dict.npy', item_dict)
np.save('/data/private/Arena/prepro_results/item_list.npy', item_list)
np.save('/data/private/Arena/prepro_results/item2keywd.npy', item2keywd)

In [ ]:
keyword_dict = {}
for i, keyword in enumerate(keyword_list):
    keyword_dict[keyword] = i
    
np.save('./prepro_results/keywrod_dict.npy', keyword_dict)
np.save('./prepro_results/keywrod_list.npy', keyword_list)

In [ ]:
len(keyword_list)

In [60]:
id2writer = ['unk']
id2reader = ['unk']
for data in tqdm_notebook(users_list):
    id2reader.append(data['id'])
    for writer in data['following_list']:
        if writer not in id2writer:
            id2writer.append(writer)

In [62]:
reader2id = {}
for i, reader in enumerate(id2reader):
    reader2id[reader] = i
    
np.save('/data/private/Arena/prepro_results/id2reader.npy', id2reader)
np.save('/data/private/Arena/prepro_results/reader2id.npy', reader2id)

In [16]:
len(id2writer)

16905

In [66]:
added_writer = 0
for data in tqdm_notebook(metadata_list):
    if data['user_id'] not in id2writer:
        added_writer += 1
        id2writer.append(data['user_id'])
print(added_writer) # 0 없음

writer2id = {}
for i, writer in enumerate(id2writer):
    writer2id[writer] = i
    
np.save('/data/private/Arena/prepro_results/id2writer.npy', id2writer)
np.save('/data/private/Arena/prepro_results/writer2id.npy', writer2id)


0


In [10]:
import numpy as np
from tqdm import tqdm_notebook
import torch

id2writer = np.load('/data/private/Arena/prepro_results/id2writer.npy')
writer2id = np.load('/data/private/Arena/prepro_results/writer2id.npy', allow_pickle=True).item()
keyword_dict = np.load('/data/private/Arena/prepro_results/keyword_dict.npy', allow_pickle=True).item()
keyword_list = np.load('/data/private/Arena/prepro_results/keyword_list.npy')
keyword_dict['없음'] = len(keyword_list)
keyword_list = list(keyword_list)
keyword_list.append('없음')
valid_writer_keywd = []


for data in tqdm_notebook(metadata_list):
    user_id = writer2id[data['user_id']]
    keyword = keyword_dict[data['keyword_list'][0]] if data['keyword_list'] is True else keyword_dict['없음']
        
    valid_writer_keywd.append([user_id, keyword])

valid_writer_keywd = torch.from_numpy(np.array(valid_writer_keywd))
torch.save(valid_writer_keywd, '/data/private/Arena/prepro_results/valid_writer_keywd.pkl')
#np.save('/data/private/Arena/prepro_results/item_dict.npy', item_dict)

In [27]:
item_dict = np.load('/data/private/Arena/prepro_results/item_dict.npy', allow_pickle=True).item()
item_list = np.load('/data/private/Arena/prepro_results/item_list.npy')

writerid2items = {}
for data in tqdm_notebook(metadata_list):
    user_id = writer2id[data['user_id']]
    id_ = item_dict[data['id']]
    keyword = keyword_dict[data['keyword_list'][0] if data['keyword_list'] is True else '없음']
    
    if writerid2items.get(user_id) == None:
        writerid2items[user_id] = [[id_, user_id, keyword]]
    else:
        writerid2items[user_id].append([id_, user_id, keyword])
        
np.save('/data/private/Arena/prepro_results/writerid2items.npy', writerid2items)


In [38]:
read_path = '/data/private/Arena/datasets/read/'
item_freq = {}

for read_file in tqdm_notebook(os.listdir(read_path)):
    file = open(read_path+read_file, 'r')
    try:
        data_ = file.readlines()
    except:
        print(read_file)
    for line in data_:
        items = line.split(' ')[1:-1]
        for item in items:
            if item_freq.get(item) == None:
                item_freq[item] = 1
            else:
                item_freq[item] += 1

.2019010120_2019010121.un~
.2018111912_2018111913.swp



In [45]:
sorted_item_freq = sorted(item_freq.items(), key=lambda kv: kv[1])
[x[0] for x in sorted_item_freq[:-100:-1]]

['@brunch_141',
 '@brunch_151',
 '@brunch_145',
 '@tenbody_1305',
 '@intlovesong_28',
 '@dailylife_207',
 '@hyehyodam_19',
 '@steven_179',
 '@brunch_140',
 '@sangheeshyn_66',
 '@brunch_142',
 '@deckey1985_51',
 '@conbus_43',
 '@sweetannie_145',
 '@dailylife_219',
 '@tenbody_1164',
 '@seochogirl_1',
 '@brunch_144',
 '@x-xv_19',
 '@honeytip_945',
 '@brunch_147',
 '@brunch_105',
 '@conbus_35',
 '@mightysense_8',
 '@honeytip_940',
 '@tenbody_1297',
 '@shanghaiesther_46',
 '@brunch_143',
 '@studiocroissant_43',
 '@chofang1_15',
 '@anti-essay_133',
 '@brunch_149',
 '@brunch_148',
 '@brunch_133',
 '@nareun_143',
 '@seochogirl_6',
 '@peregrino97_779',
 '@brunch_2',
 '@bzup_281',
 '@brunch_1',
 '@noey_130',
 '@zorbayoun_14',
 '@kam_60',
 '@anti-essay_124',
 '@mightysense_9',
 '@seochogirl_16',
 '@kam_33',
 '@dailylife_173',
 '@seochogirl_18',
 '@minimalmind88_26',
 '@wootaiyoung_85',
 '@jmg5308_163',
 '@seochogirl_7',
 '@seochogirl_8',
 '@seochogirl_17',
 '@seochogirl_3',
 '@wikitree_54',
 '@ps

In [54]:
userid2followid = {}
for data in users_list:
    id_ = reader2id[data['id']]
    following_list = [writer2id[x] for x in data['following_list']]
    userid2followid[id_] = following_list
    
np.save('/data/private/Arena/prepro_results/userid2followid.npy', userid2followid)

In [56]:
item_dict['@awesomedaytoeat_8']

KeyError: '@awesomedaytoeat_8'